In [1]:
import warnings
import sys, getopt
import pandas as pd
import numpy as np
import os
import random
import time
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics.pairwise import euclidean_distances
from dependencies.cluster_cells import ssKMeans
from dependencies.helper_functions import *
import subprocess

warnings.filterwarnings("ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'dependencies'

In [5]:
pd.read_csv('../raw_data_nottoupload/mgDF.csv', index_col=0)

/home/aanchal/.conda/envs/aanchalPy3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CenterX,CenterY,HLA.ABC,CD57,CD31,CD14,C.peptide,Nestin,Glucagon,Keratin,...,zs_CD3,zs_pS6,zs_CD45RO,zs_HLA.DR,zs_PP,zs_Ghrelin,zs_DNA191,zs_DNA193,TIFFfilename,Status
1,-493.762,496.012,3.34108,0.138356,0.146041,5.11426,0.172109,0.396432,0.984216,8.252990,...,-0.049842,0.543564,0.217285,-0.181882,-0.290428,-0.059119,-0.586171,-0.409180,HPAP002_1_C_head_ROI1.tif,T1D
2,-483.230,493.446,2.25562,0.045045,0.045045,3.31649,0.072072,0.216198,0.491336,4.071650,...,-1.010599,-0.030196,-0.498005,-0.223103,-0.295474,-0.053326,-1.196808,-1.155048,HPAP002_1_C_head_ROI1.tif,T1D
3,-458.603,497.500,4.27949,0.000000,0.165440,4.05684,0.000000,0.102941,0.297435,1.025580,...,-1.392878,-0.563533,-0.214109,0.323349,-0.295333,-0.057557,-1.192922,-1.206121,HPAP002_1_C_head_ROI1.tif,T1D
4,-437.266,496.864,4.60388,0.012987,0.058256,2.14239,0.000000,0.255810,0.025974,0.406695,...,1.941695,-0.080725,2.821473,0.378144,-0.304778,-0.059288,-0.056559,-0.085184,HPAP002_1_C_head_ROI1.tif,T1D
5,-260.069,496.552,2.47759,0.072484,0.089654,4.57637,0.000000,0.034483,1.059540,6.208520,...,-1.108237,-0.427888,-0.355091,-0.288095,-0.302500,-0.066359,-1.101903,-1.014889,HPAP002_1_C_head_ROI1.tif,T1D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176667,559.000,-498.250,5.55450,0.000000,0.208333,3.63800,0.924366,0.624323,1.397150,6.147780,...,0.191726,0.420836,0.251243,-0.077530,0.083784,-0.162074,0.541382,0.980853,HPAP-026_IMC_Tail-of-pancreas_ROI3_ROI003_ROI ...,GAD+
1176668,-343.343,-498.243,4.71677,0.430101,0.171429,2.68909,0.057143,0.646506,1.162230,5.170220,...,0.036446,-0.260918,-0.275857,-0.597759,-0.069326,-0.010891,-0.358843,-0.295634,HPAP-026_IMC_Tail-of-pancreas_ROI3_ROI003_ROI ...,GAD+
1176669,-313.065,-498.371,6.29571,0.396005,0.313993,10.23660,0.362787,0.908652,0.516129,2.994820,...,-0.118575,0.645405,0.744544,0.050481,-0.110648,-0.001184,0.932020,0.803530,HPAP-026_IMC_Tail-of-pancreas_ROI3_ROI003_ROI ...,GAD+
1176670,-406.231,-499.038,4.91625,0.000000,0.230769,2.63169,0.153846,1.328370,1.069200,2.908870,...,-1.075291,0.124947,0.025918,-0.466963,-0.217217,0.028683,-0.511086,-0.423289,HPAP-026_IMC_Tail-of-pancreas_ROI3_ROI003_ROI ...,GAD+


In [26]:
def anno():
    #def main(argv):

    #compulsory
    # data_path = '' #'data/mgDF.csv'
    # signature_path= '' #'data/signatures_T1D.csv'
    # op_dir = 'output_AnnoSpat'
    # p1 = 'HLA.ABCp' # first protein/column in protein expression file
    # p2 = 'Ghrelinp' # last protein/column in protein expression file
    # roi_name_col = 'TIFFfilename'

    #optional
    toDrop=''
    suffix = '_IMC_T1D_AnnoSpat'
    disease_status_col = '' # disease_status_col = 'Status'
    classifier = 'ELM'#
    th=[99.9, 99.999, 70] 
    adaptive_th_vec=[99.5,99.5,99.5,99.5,99.9,  99,99.5,  99,99,  99.5,99.9,  99.9,99.9,99.9,  99.5,99.5] # 2 adaptive_th_vec

    #fixed
    sampling_ratio=0.5 # train test ratio (50% train ROIs labelled using semi supervised clustering, remaining test ROIs labelled by training classifier )


    try:
        opts, args = getopt.getopt(sys.argv[1:],"hi:m:o:f:l:s:r:d:c:t:a:",["inputfile=","markerfile=","outputdir=","firstprotein=","lastprotein=","suffix=","roicol=","diseasecol=","classifier=","thresholdmaxandunknown=","thresholdadaptive="])
    except getopt.GetoptError:
        print ('Error \n python3 classify_IMCcells.py -i <inputfile> -m <signaturefile> -o <outputdir> -f <firstprotein> -l <lastprotein> -s <suffix> -r <roicol> -d <diseasecol> -c <classifier> -t <thresholdmaxandunknown> -a <thresholdadaptive>')
        sys.exit(2)

    for opt, arg in opts:
        if  opt in ("-h", "--help"):
            print ('python3 classify_IMCcells.py -i <inputfile> -m <marlerfile> -o <outputdir> -f <firstprotein> -l <lastprotein> -s <suffix> -r <roicol> -d <diseasecol> -c <classifier> -t <thresholdmaxandunknown> -a <thresholdadaptive>')
            sys.exit()

        elif opt in ("-i", "--inputfile"):
            data_path = arg
        elif opt in ("-m", "--markerfile"):
            signature_path = arg
        elif opt in ("-o", "--outputdir"):
            op_dir = arg
        elif opt in ("-f", "--firstprotein"):
            p1 = arg
        elif opt in ("-l", "--lastprotein"):
            p2 = arg
        elif opt in ("-s", "--suffix"):
            suffix = arg
        elif opt in ("-r", "--roicol"):
            roi_name_col  = arg
        elif opt in ("-d", "--diseasecol"):
            disease_status_col = arg
        elif opt in ("-c", "--classifier"):
            classifier = arg
        elif opt in ("-t", "--thresholdmaxandunknown"):
            th_str = arg[1: len(arg) -1].split(',')
            th = [float(item) for item in th_str]

        elif opt in ("-a", "--thresholdadaptive"):
            adaptive_th_vec_str = arg[1: len(arg) -1].split(',')
            adaptive_th_vec = [float(item) for item in adaptive_th_vec_str]

    #print ('Argument List:', str(sys.argv))

    print('\nStarting cell type annotation.....')




    if not os.path.exists(op_dir):
        os.makedirs(op_dir)

    #### input: signature/marker file and single cell data (having protien expression, ROI_names, disease codnition)

    raw_data_file = pd.read_csv(data_path, index_col=0) # contains "cells" on rows and "proteins", "ROI_name","disease status" (optional), cell coordinates (optional) on columns
    signatures=pd.read_csv(signature_path , index_col=0) 

    # # IMC-T1D
    logtr=1 
    un=1 

    raw_data = raw_data_file.loc[:, p1:p2]# data_imct1d.transpose()
    if (len(toDrop) !=0):
        raw_data.drop(toDrop, axis =1, inplace =True)

    # get ROI name for each cell
    roi_names_allcells = raw_data_file.loc[:, roi_name_col]
    if( len(disease_status_col) == 0):
        roi_names_status_allcells = raw_data_file.loc[:, [roi_name_col]]
    else:
        roi_names_status_allcells = raw_data_file.loc[:, [roi_name_col,disease_status_col]]


    unique_rois = roi_names_status_allcells.drop_duplicates()

    unique_rois = roi_names_status_allcells.drop_duplicates()

    if(len(disease_status_col) != 0):
        noDiseaseConditions= len(unique_rois[disease_status_col ].unique() )
    else:
        noDiseaseConditions=1

    if( (len(disease_status_col) == 0) | (noDiseaseConditions ==1) ):
        tr_rois, te_rois = train_test_split(unique_rois[roi_name_col].values,test_size = 1-sampling_ratio, random_state=0)
    else:
        tr_rois, te_rois, tr_status, te_status = train_test_split(unique_rois[roi_name_col].values,unique_rois[disease_status_col ].values,test_size = 1-sampling_ratio,  stratify = unique_rois[disease_status_col ].values, random_state=0)

    raw_data_tr = raw_data.loc[ roi_names_allcells.isin(tr_rois) ]
    raw_data_te = raw_data.loc[ roi_names_allcells.isin(te_rois) ]

    dict_celltypes= { ii : signatures.columns[ii] for ii in range(len(signatures.columns)) }
    dict_celltypes.update({ len(signatures.columns): 'Unknown'})

    #np.save('../Python_objects/CTmap'+suffix+'.npy', dict_celltypes) 
    #dict_celltypes = np.load('../Python_objects/CTmap'+suffix+'.npy',allow_pickle='TRUE').item()

    imc_forSSC_raw=raw_data_tr.copy()
    imc_forSSC_raw.index=np.arange(len(imc_forSSC_raw) ) #to avoid descripency in initial labelling

    #### pre-processing data

    # #Scale input vectors individually to unit norm (vector length) so that euclidean dist in high dim space correlates with cosine distance (cells/samples lie on a unit sphere in the feature/proteomic space; direction of cells in proteomic space is more important than the magnitutde; take and example of 4 cells in 2D proteomic space)
    imc_forSSC_log =  imc_forSSC_raw
    if (logtr==1):
        imc_forSSC_log = pd.DataFrame( np.log(1+imc_forSSC_raw), index=imc_forSSC_raw.index,columns= imc_forSSC_raw.columns )


    imc_forSSC=pd.DataFrame(imc_forSSC_log, index=imc_forSSC_raw.index,columns= imc_forSSC_raw.columns )
    if(un==1):
        #log tr data (reduce the effect of outliers)
        #imc_forSSC=pd.DataFrame(  imc_forSSC, index=imc_forSSC.index,columns= imc_forSSC.columns ) #less accurate than log tr in terms of visulaization
        imc_forSSC=pd.DataFrame( preprocessing.normalize(imc_forSSC_log, norm='l2'), index=imc_forSSC_raw.index,columns= imc_forSSC_raw.columns )


    #### initialize

    k=signatures.shape[1] +1 #(1 extra cluster for unknown cell type)

    # Initialise centroids  
    centroids = np.zeros((k , imc_forSSC.shape[1]) , dtype=object)

    # Create a list to store which class/cell-type-centroid is assigned to each sample/label vector
    assigned_centroids = np.zeros(len(imc_forSSC), dtype = np.int32)-1

    #### get order of cell type abundance

    dec_abudance_ind=get_cell_type_abundance(imc_forSSC_raw , signatures)

    #### Initial labelling to get the supervised samples

    (assigned_centroids,centroids ) = initial_labelling(assigned_centroids, centroids, imc_forSSC, signatures, dec_abudance_ind, th,adaptive_th_vec)

    write_initial_labels(assigned_centroids , raw_data_tr.index,raw_data_te.index,op_dir, suffix)

    tic = time.time()

    #kmeans = ssKMeans(k=k )

    # kmeans.fit( imc_forSSC.values , centroids, assigned_centroids)

    # with open('models/model_kmeans', 'wb') as dictionary_file:
    #     pickle.dump(kmeans, dictionary_file)
    with open('models/model_kmeans', 'rb') as dictionary_file:
        kmeans = pickle.load(dictionary_file)

        print("Estimating cell types...")
    toc=time.time()
    #print("Time taken for first half: "+(toc - tic)/60)

    suffix

    pd.DataFrame( kmeans.cluster_centers_ , columns = imc_forSSC.columns).to_csv(op_dir+'/centroids_'+suffix+'.csv', index=False)

    assigned_labels=pd.DataFrame(kmeans.labels_)#.to_csv()
    assigned_labels.index=raw_data_tr.index

    (tr_labels,tr_labels_numericLabels) = process_cell_labels(assigned_labels,dict_celltypes,raw_data_tr.index)

    ### Train classifer

    use_trained_model =0 # 0 if predicting first time, 1 if not

    data_tr = raw_data_tr.copy() #required for training classifiers
    data_te =raw_data_te.copy()

    # classify test cells (may take about a minute)
    te_labels = classify_test_cells(classifier,data_tr, tr_labels_numericLabels['label'], data_te )

    predictNsave(classifier,suffix,op_dir, te_labels, raw_data_te.index ,tr_labels, tr_labels_numericLabels, raw_data_tr.index, dict_celltypes )

    # saved labels/cell type annotations in 
    print("Annotations saved in: "+op_dir+'/trte_labels_numericLabels_'+classifier+suffix+'.csv')

'output_AnnoSpat/trte_labels_numericLabels_ELM_IMC_T1D_AnnoSpat.csv'

In [19]:
### spatial analysis


def spatial(op_dir, classifier, suffix):
    command = 'Rscript'
    path2script = 'sample_Rscript.R'
    args = [op_dir+'/trte_labels_'+classifier+suffix+'.csv']
    cmd = [command, path2script] + args
    cmd
    print("printing the sytem command:",cmd)
    x = subprocess.check_output(cmd, universal_newlines=True)
    print('The #rows and cols is (from R script):', x)

The #rows and cols is (from R script): ---R script running-------
1170001
2

